<a href="https://colab.research.google.com/github/barrylunemann/mapping/blob/main/folium_location_markers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
# A script to create an interactive map with icons showing specific locations

# Your Excel file must have a column named "Latitude", "Longitude", "Icon Name",
# "Icon Color"

# Icon Color must be one of the following.
# [‘red’, ‘blue’, ‘green’, ‘purple’, ‘orange’, ‘darkred’,
# ’lightred’, ‘beige’, ‘darkblue’, ‘darkgreen’, ‘cadetblue’,
# ‘darkpurple’, ‘white’, ‘pink’, ‘lightblue’, ‘lightgreen’,
# ‘gray’, ‘black’, ‘lightgray’]

# "Location Name"

In [49]:
!pip install geopandas
import folium
import pandas as pd
import geopandas as gpd

In [50]:
# Use this to mount your Google Drive to the script.
# When this line runs, it will ask for permission to access your drive.
# I will not have access to this, it will only give the session you are in
# access to your drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Here is where you enter your inputs.**

In [51]:
# Here is where you would enter your inputs.

# Starting Coordinates of the map. I entered the default
# as a zoomed out view of the USA. You can use Google Maps to find the
# center of whatever you would like it centered on.
# USA Center = (39.131324555683335, -94.75295974259164)
starting_coordinates = (44.971305140498764, -93.27625946683504)

# How zoomed in do you want it?
zoom_selection = 9

# Import your excel file (reminder you need Zip Code and Zip Code Score)
# You will need to copy the link location to your own Google Drive
main_file = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Data/Medical Device Company Locations.xlsx',
                          sheet_name='Sheet1')

# Tool tip pop-ups on the zip codes. If you have other columns in your Excel
# file that you would like to show up when you click on or hover over the icon.
# This is where you need to list them.
column_list = ['Location Name',
               'Address']

# Name your output file
output_filename = "Output File Icons"

In [52]:
# Creating the map object
sweet_new_map = folium.Map(starting_coordinates,
                           tiles='openstreetmap',
                           zoom_start=zoom_selection)

In [53]:
# Creating Object for the map
feature_group = folium.FeatureGroup(name='Icons', show=True)

# Filling any blanks that may have been left
main_file.fillna('', inplace=True)

In [54]:
# Creating html popup and hover text
def html_text_creator(df, col_list):
  str_name = ""
  # print(col_list)
  #print(df.loc['Location Name'])
  for loop in col_list:
    # print(loop)
    try:
      if len(str_name) == 0:
        # print(df.loc[loop])
        str_name += '<b>' + loop + ':</b> ' + df.loc[loop]
      else:
        # print(df.loc[loop])
        str_name += '<br><b>' + loop + ':</b> ' + df.loc[loop]
    except Exception:
      print("Error")
  return(str_name)


main_file['html_text'] = main_file.apply(lambda row:
                                         html_text_creator(row, column_list),
                                         axis=1)


In [62]:
print(main_file.iloc[6, 6])
print(main_file.iloc[7, 6])


<b>Location Name:</b> Medtronic - Mounds View<br><b>Address:</b> 8200 Coral Sea St NE, Mounds View, MN 55112
<b>Location Name:</b> Medtronic Croventis<br><b>Address:</b> 1410 Energy Park Dr, St Paul, MN 55108


In [67]:
# Looping through the list
for iter in range(len(main_file)):
  html_message = (main_file.loc[iter, 'html_text'])
  popup = folium.Popup(html_message,
                       max_width=500)
  tooltip_msg = (main_file.loc[iter, 'html_text'])
  folium.Marker(
                location=[main_file.loc[iter, 'Latitude'],
                          main_file.loc[iter, 'Longitude']],
                tooltip=tooltip_msg,
                popup=popup,
                icon=folium.Icon(color=main_file.loc[iter, 'Icon Color'],
                                 icon=main_file.loc[iter, 'Icon Name'],
                                 prefix='fa')
                ).add_to(feature_group)
        
# Adding to mapping object
sweet_new_map.add_child(feature_group)
sweet_new_map.add_child(folium.map.LayerControl(collapsed=False))

In [56]:
#Creating your output file
sweet_new_map.save(output_filename+'.html')